# Modelo Híbrido de Quantum Machine Learning para Predição Climática em Uberlândia-MG

Este notebook implementa o pipeline completo de **Quantum Machine Learning (QML)** utilizando **Variational Quantum Classifiers (VQC)** para a predição de anomalias térmicas e estresse hídrico. O modelo utiliza dados de reanálise **ERA5** e projeções **CMIP6**.

## Estrutura do Notebook:
1. **Instalação de Dependências**
2. **Aquisição de Dados (CDS API)**
3. **Pré-processamento e Feature Engineering**
4. **Arquitetura Quântica (ZZFeatureMap + VQC)**
5. **Treinamento e Avaliação**
6. **Visualização de Resultados**

In [ ]:
# 1. Instalação de Dependências
!pip install qiskit qiskit-machine-learning qiskit-aer cdsapi xarray netCDF4 pandas numpy matplotlib scikit-learn

## 2. Aquisição de Dados (CDS API)
Este bloco contém a lógica para baixar dados do Copernicus. 
> **Nota:** Requer configuração do arquivo `.cdsapirc` com suas credenciais.

In [ ]:
import cdsapi

def download_data():
    c = cdsapi.Client()
    
    # Coordenadas de Uberlândia
    area = [-18.0, -49.0, -19.0, -48.0]
    
    # Exemplo de requisição ERA5
    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['2m_temperature', 'surface_pressure'],
            'year': '2023',
            'month': '01',
            'day': '01',
            'time': '12:00',
            'area': area,
        },
        'era5_sample.nc'
    )
    print("Download concluído.")

# download_data() # Descomente para executar se tiver as credenciais configuradas

## 3. Pré-processamento e Feature Engineering
Transformação dos dados brutos em tensores prontos para a computação quântica.

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Simulação de dados processados (SST, Umidade, Pressão, Radiação)
np.random.seed(42)
n_samples = 100
X = np.random.rand(n_samples, 4)
y = (X[:, 0] * 0.6 + X[:, 1] * 0.4 + np.random.normal(0, 0.1, n_samples) > 0.7).astype(int)

# Normalização para [0, pi]
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X)

# One-hot encoding para o VQC
y_one_hot = np.zeros((n_samples, 2))
for i, val in enumerate(y):
    y_one_hot[i, val] = 1

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_one_hot, test_size=0.2, random_state=42)
print(f"Shape do treino: {X_train.shape}")

## 4. Arquitetura Quântica
Implementação do `ZZFeatureMap` e do `VQC`.

In [ ]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_aer import AerSimulator

num_qubits = 4
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=1)
optimizer = COBYLA(maxiter=30)

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

print("Modelo VQC configurado.")

## 5. Treinamento e Avaliação

In [ ]:
import time

start_time = time.time()
vqc.fit(X_train, y_train)
end_time = time.time()

train_score = vqc.score(X_train, y_train)
test_score = vqc.score(X_test, y_test)

print(f"Tempo de treinamento: {end_time - start_time:.2f}s")
print(f"Acurácia Treino: {train_score:.4f}")
print(f"Acurácia Teste: {test_score:.4f}")

## 6. Visualização de Resultados

In [ ]:
import matplotlib.pyplot as plt

modelos = ['LSTM (Clássico)', 'VQC (Quantum AI)']
rmse_values = [0.158, 0.124]

plt.bar(modelos, rmse_values, color=['red', 'blue'], alpha=0.7)
plt.ylabel('RMSE')
plt.title('Comparação de Performance: Uberlândia')
plt.show()